In [419]:
from deta import Deta
import pandas as pd
import numpy as np
import yfinance as yf
import datetime as dt

In [13]:
class StockyDb:
    def __init__(self):
        self.deta = Deta('d0p5if1f_GSnmoPk32YPhwKaJzN6sq7hM2DN4XPks')
        self.db = self.deta.Base('StockyAI')
        
    def save_to_DB(self,ticker,df):
        date_=[]
        open_=[]
        high_=[]
        low_=[]
        close_=[]
        for i in range (0,len(df)):
            df['Close']=df['Close'].astype(float)
            df['Low']=df['Low'].astype(float)
            df['High']=df['High'].astype(float)
            df['Open']=df['Open'].astype(float)
            date_.append(df['Date'][i])
            open_.append(df['Open'][i])
            high_.append(df['High'][i])
            low_.append(df['Low'][i])
            close_.append(df['Close'][i])
        self.db.put({'key':ticker,'date':date_,'open':open_,'high':high_,'low':low_,'close':close_})
    
    def ticker_df(self,ticker):
        return self.db.get(key=ticker)
    
    def get_all_ticker(self):
        tickers=[]
        for i in range(0,self.db.fetch().count):
            tickers.append(self.db.fetch().items[i]['key'])
        return tickers
    
    def Recomodation(self):
    tickers = DB.get_all_ticker()
    buy=[]
    sell = []
    
    for i in tickers:
        rec={}
        df=pd.DataFrame(DB.ticker_df(i))
        H_max=df['high'].max()
        L_min=df['low'].min()
        H_date=(df[df['high']==H_max])['date'].to_list()
        L_date=(df[df['low']==L_min])['date'].to_list()
        if H_date>L_date:
            price={}
            price['Ticker']=i
            price['L_buy_price']=L_min
            price['H_buy_price']=L_min+(L_min*0.01)
            price['H_sell_price']=H_max
            price['L_sell_price']=H_max-(H_max*0.01)
            price['profit% ']=((price['L_sell_price']-price['H_buy_price'])/price['H_buy_price'])*100
            buy.append(price)
        elif H_date<L_date:
            price={}
            price['Ticker']=i
            price['H_sell_price']=H_max
            price['L_sell_price']=H_max-(H_max*0.01)
            price['L_buy_price']=L_min
            price['H_buy_price']=L_min+(L_min*0.01)
            price['profit% ']=((price['L_sell_price']-price['H_buy_price'])/price['H_buy_price'])*100
            sell.append(price)
    
    buy_df=pd.DataFrame()
    for i in range (0,len(buy)):
        buy_df=buy_df.append([buy[i]],ignore_index=True)
    sell_df=pd.DataFrame()
    for j in range (0,len(sell)):
        sell_df=sell_df.append([sell[j]],ignore_index=True)
    return buy_df,sell_df

In [161]:
dbp = deta.Base('StockyAI_portfolio')

In [392]:
class Portfolio():
    def __init__(self):
        self.dbp = deta.Base('StockyAI_portfolio')
        self.p_data = self.dbp.get(key='praveen')
        self.cash = self.p_data['cash']
        self.stocks = self.p_data['stocks']
        #self.S_df = pd.DataFrame(columns=['stock','buy_price','quantity'])
        #self.S_df=self.S_df.append(self.stocks,ignore_index=True)
        print(self.stocks)
        
    def Buy (self,ticker):
        tik= ticker
        ticker = yf.Ticker(ticker)
        price = ticker.info
        if (price['regularMarketPrice']) == None :
            pre_day_price=ticker.history(period='1d')
            print(ticker,"Price - ",pre_day_price.iloc[0,3])
            c_price =pre_day_price.iloc[0,3]
            quant= int(input('Quantity'))
            amt = c_price*quant
            if self.cash >= amt:
                self.cash=self.cash-amt
                S_detail={}
                S_detail['buy_price']=c_price
                S_detail['quantity'] = quant
                self.stocks[tik]=S_detail
                #return self.stocks
                self.dbp.put({'key':'praveen','cash':self.cash,'stocks':self.stocks})
                
            else:
                print('INSUFICENT BALANCE')

        else:
            print("Price - ",price['regularMarketPrice'])
            c_price =price['regularMarketPrice']
            quant= int(input('Quantity'))
            amt = c_price*quant
            if self.cash >= amt:
                self.cash=self.cash-amt
                print('stocks brought')
                print('available balance - ',cash)
            else:
                print('INSUFICENT BALANCE')

    def Sell (self,ticker):
        tik=ticker
        ticker = yf.Ticker(ticker)
        price = ticker.info
        if (price['regularMarketPrice']) == None :
            pre_day_price=ticker.history(period='1d')
            print(ticker,"Price - ",pre_day_price.iloc[0,3])
            c_price =pre_day_price.iloc[0,3]
            quant= int(input('Quantity'))
            amt = c_price*quant
            if self.stocks[tik]['quantity']>= quant :
                self.cash=self.cash+amt
                r_quant = self.stocks[tik]['quantity']-quant
                S_details=(self.stocks[tik])
                S_details['quantity']=r_quant
                self.stocks[tik]=S_details
                #print(self.stocks[tik])
                #return self.stocks
                self.dbp.put({'key':'praveen','cash':self.cash,'stocks':self.stocks})
                
                
            else:
                print('INSUFICENT BALANCE')

        else:
            print("Price - ",price['regularMarketPrice'])
            c_price =price['regularMarketPrice']
            quant= int(input('Quantity'))
            amt = c_price*quant
            if cash >= 0:
                cash=cash+amt
                print('stocks Sold')
                print('available balance - ',cash)
            else:
                print('INSUFICENT BALANCE')
        
    def get_holdings(self):
        #print(self.p_data['stocks'])
        hold_df = pd.DataFrame(self.p_data['stocks'])
        hold_df=hold_df.transpose()
        hold_df=hold_df.reset_index()
        hold_df['index']=hold_df['index'].astype('str')
        tiks=hold_df['index'].to_list()
        #tickers=yf.Tickers(tiks)
        prices=[]
        
        for i in (tiks):
            #i = tickers.tickers[i]
            pr = get_live_price(i)
            #pr = pr['regularMarketPrice']
            prices.append(pr)
            
        #hold_df['current price']=hold_df['index'].apply(self.get_current_price)
        hold_df['current price']=prices
        return hold_df
    
    def get_current_price(self,ticker):
        ticker=yf.Ticker(ticker)
        info=ticker.info
        return info['regularMarketPrice']

In [393]:
prot = Portfolio()

{'INDUSINDBK.NS': {'buy_price': 1147.8499755859375, 'quantity': 10}, 'RELIANCE.NS': {'buy_price': 2502.199951171875, 'quantity': 0}, 'SBIN.NS': {'buy_price': 574, 'quantity': 90}}


In [394]:
holding=prot.get_holdings()

In [395]:
holding

,index,buy_price,quantity,current price
0,INDUSINDBK.NS,1147.849976,10.0,1147.849976
1,RELIANCE.NS,2502.199951,0.0,2502.199951
2,SBIN.NS,574.000000,90.0,574.000000


In [274]:
def get_current_price(ticker):
    ticker=yf.Ticker(ticker)
    info=ticker.info
    return info['regularMarketPrice']

In [275]:
get_current_price('INDUSINDBK.NS')

In [389]:
from yahoo_fin.stock_info import *

In [402]:
print(get_live_price('ADANIENT.NS'))

3642.199951171875


In [399]:
N50_list=tickers_nifty50()

In [414]:
N50_price = {}
for i in (N50_list):
    print(i)
    if i == 'MM.NS':
        N50_price[i]=get_live_price('M&M.NS')
    else:
        N50_price[i]=get_live_price(i)

ADANIENT.NS
APOLLOHOSP.NS
BAJAJ-AUTO.NS
BAJAJFINSV.NS
BAJFINANCE.NS
BHARTIARTL.NS
BRITANNIA.NS
CIPLA.NS
COALINDIA.NS
HDFCLIFE.NS
HEROMOTOCO.NS
HINDALCO.NS
ICICIBANK.NS
INDUSINDBK.NS
ITC.NS
KOTAKBANK.NS
LT.NS
MARUTI.NS
MM.NS
NESTLEIND.NS
NTPC.NS
ONGC.NS
RELIANCE.NS
TATACONSUM.NS
TATASTEEL.NS
TCS.NS
TECHM.NS
TITAN.NS
ULTRACEMCO.NS
WIPRO.NS


In [415]:
N50_price

{'ADANIENT.NS': 3642.199951171875,
 'APOLLOHOSP.NS': 4700.14990234375,
 'BAJAJ-AUTO.NS': 3541.800048828125,
 'BAJAJFINSV.NS': 1497.050048828125,
 'BAJFINANCE.NS': 6374.60009765625,
 'BHARTIARTL.NS': 808.6500244140625,
 'BRITANNIA.NS': 4331.89990234375,
 'CIPLA.NS': 1119.1500244140625,
 'COALINDIA.NS': 215.0500030517578,
 'HDFCLIFE.NS': 565.4500122070312,
 'HEROMOTOCO.NS': 2636.85009765625,
 'HINDALCO.NS': 429.54998779296875,
 'ICICIBANK.NS': 878.9000244140625,
 'INDUSINDBK.NS': 1147.8499755859375,
 'ITC.NS': 326.70001220703125,
 'KOTAKBANK.NS': 1821.949951171875,
 'LT.NS': 2062.75,
 'MARUTI.NS': 8141.60009765625,
 'MM.NS': 1224.5999755859375,
 'NESTLEIND.NS': 20132.75,
 'NTPC.NS': 163.0,
 'ONGC.NS': 139.8000030517578,
 'RELIANCE.NS': 2502.199951171875,
 'TATACONSUM.NS': 779.0,
 'TATASTEEL.NS': 102.25,
 'TCS.NS': 3228.35009765625,
 'TECHM.NS': 995.75,
 'TITAN.NS': 2483.050048828125,
 'ULTRACEMCO.NS': 6909.39990234375,
 'WIPRO.NS': 377.6499938964844}

,open,high,low,close,adjclose,volume,ticker
2002-08-12,508.100006,512.549988,505.000000,505.149994,413.536560,7100.0,NESTLEIND.NS
2002-08-13,503.500000,515.000000,503.500000,509.049988,416.729309,8800.0,NESTLEIND.NS
2002-08-14,502.100006,515.000000,502.100006,514.400024,421.108917,18700.0,NESTLEIND.NS
2002-08-15,514.400024,514.400024,514.400024,514.400024,421.108917,0.0,NESTLEIND.NS
2002-08-16,516.000000,516.849976,510.000000,511.000000,418.325592,9400.0,NESTLEIND.NS
...,...,...,...,...,...,...,...
2022-12-19,19800.000000,20120.000000,19799.949219,20081.650391,20081.650391,65448.0,NESTLEIND.NS
2022-12-20,20150.000000,20167.849609,19851.050781,20135.550781,20135.550781,46676.0,NESTLEIND.NS
2022-12-21,20165.000000,20348.000000,20062.150391,20314.199219,20314.199219,70378.0,NESTLEIND.NS
2022-12-22,20395.000000,20440.000000,20157.500000,20306.599609,20306.599609,37688.0,NESTLEIND.NS


In [412]:
get_live_price()

NameError: name 'f' is not defined

In [416]:
dbh=deta.Base('StockyAI_home')

In [418]:
dbh.put({'key':'closing price','price':N50_price})

{'key': 'closing price',
 'price': {'ADANIENT.NS': 3642.199951171875,
  'APOLLOHOSP.NS': 4700.14990234375,
  'BAJAJ-AUTO.NS': 3541.800048828125,
  'BAJAJFINSV.NS': 1497.050048828125,
  'BAJFINANCE.NS': 6374.60009765625,
  'BHARTIARTL.NS': 808.6500244140625,
  'BRITANNIA.NS': 4331.89990234375,
  'CIPLA.NS': 1119.1500244140625,
  'COALINDIA.NS': 215.0500030517578,
  'HDFCLIFE.NS': 565.4500122070312,
  'HEROMOTOCO.NS': 2636.85009765625,
  'HINDALCO.NS': 429.54998779296875,
  'ICICIBANK.NS': 878.9000244140625,
  'INDUSINDBK.NS': 1147.8499755859375,
  'ITC.NS': 326.70001220703125,
  'KOTAKBANK.NS': 1821.949951171875,
  'LT.NS': 2062.75,
  'MARUTI.NS': 8141.60009765625,
  'MM.NS': 1224.5999755859375,
  'NESTLEIND.NS': 20132.75,
  'NTPC.NS': 163,
  'ONGC.NS': 139.8000030517578,
  'RELIANCE.NS': 2502.199951171875,
  'TATACONSUM.NS': 779,
  'TATASTEEL.NS': 102.25,
  'TCS.NS': 3228.35009765625,
  'TECHM.NS': 995.75,
  'TITAN.NS': 2483.050048828125,
  'ULTRACEMCO.NS': 6909.39990234375,
  'WIPRO.N

In [443]:
now=dt.datetime.now()

In [444]:
now.strftime("%H:%M")

'23:11'

In [439]:
while now.hour == 22 :
    print('working')
    break

working
